In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-22 15:59:08--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-09-22 15:59:09 (45.8 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
!pip install -U "ai21>=2.13.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: anyio
    Found existing installation: anyio 3.7.1
    Uninstalling anyio-3.7.1:
      Su

In [3]:
import requests
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [4]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [5]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [17]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# rag: retrieval
def search(query):
    filter_dict = {"course": "data-engineering-zoomcamp"}
    boost_dict = {"question": 3, "section": 0.5}

    results = index.search(
        query,
        filter_dict,
        boost_dict
    )
    return results
search_results = search("The course has already started can i still enroll?")

In [8]:
query = "The course has already started can i still enroll?"

# rag: augmentation
def build_prompt(query,search_results):
    prompt_template="""
QUESTION: {question}

CONTEXT: {context}

ANSWER:
""".strip()
    context =""
    for doc in search_results:
        context =context+f"section: {doc['section']} \nquestion: {doc['question']}\n answer: {doc['text']} \n\n"
    prompt = prompt_template.format(question=query , context=context).strip()
    return prompt
prompt = build_prompt(query,search_results)

In [18]:
prompt

"QUESTION: The course has already started can i still enroll?\n\nCONTEXT: section: General course-related questions \nquestion: Course - Can I still join the course after the start date?\n answer: Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. \n\nsection: General course-related questions \nquestion: Course - Can I follow the course after it finishes?\n answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project. \n\nsection: General course-related questions \nquestion: Course - When will the course start?\n answer: The purpose of this document is to capture frequently asked technical questions\

In [9]:
from google.colab import userdata


In [10]:
# rag: generation
from ai21 import AI21Client
from ai21.models.chat import UserMessage
import json
# One way of passing your key to the client.
import os

# AI21_API_KEY = os.environ["AI21_API_KEY"]
AI21_API_KEY = userdata.get('AI21_API_KEY')
client = AI21Client(api_key=AI21_API_KEY)

def single_message_instruct(content):
    messages = [UserMessage(content=content)]
    response = client.chat.completions.create(
        model="jamba-1.5-large",
        messages=messages,
        top_p=1.0 # Setting to 1 encourages different responses each call.
    )
    return response.to_json()



In [19]:
# def llm(prompt):
#     response = single_message_instruct(prompt)
#     json_response = json.loads(response)
#     content = json_response["choices"][0]["message"]["content"]
#     return content
def llm(prompt,generate_params=None):
    if generate_params is None:
        generate_params = {}
    message = [ {"role": "user", "content": prompt} ]

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )
    generation_args = {
        "max_new_tokens": 500,
        "return_full_text": False,
        "temperature": 0.0,
        "do_sample": False,
    }

    output = pipe(message, **generation_args)
    result = output[0]['generated_text']
    return result

In [20]:
rag("I just discovered the course. Can i still join it?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


" Yes, you can still join the course after the start date. You are eligible to submit the homeworks even if you don't register. However, there will be deadlines for turning in the final projects, so make sure not to leave everything until the last minute."